In [58]:
import sys
import re
from pandas import read_csv
from itertools import groupby
from operator import itemgetter
from collections import Iterable
from nltk.corpus import stopwords
import NE_candidate_module as ne


In [59]:
cachedStopWords = set(stopwords.words("english"))
cachedStopWords.update(("and","i","i'll","i've","i'm","a","so","arn't","this","when","it","many","can't","cant","yes","no","these","to"))
cachedTitles = ["Mr.","Mr","Mrs.","Mrs","Miss","Ms"]
prep_list=["in","at","of","on"]

In [60]:
def printList(mylist):
    print("["),
    #print "[",
    for item in mylist:
        if item != None:
            if isinstance(item,ne.NE_candidate):
                item.print_obj()
            else:
                print (item+","),
                #print item+",",
    #print "]"
    print("]")

In [61]:
def flatten(mylist, outlist,ignore_types=(str, bytes, ne.NE_candidate)):
    
    if mylist !=[]:
        for item in mylist:
            #print not isinstance(item, ne.NE_candidate)
            if isinstance(item, list) and not isinstance(item, ignore_types):
                flatten(item, outlist)
            else:
                if isinstance(item,ne.NE_candidate):
                    item.phraseText=item.phraseText.strip(' \t\n\r')
                    item.reset_length()
                else:
                    item=item.strip(' \t\n\r')
                outlist.append(item)
    return outlist

In [62]:
def consecutive_cap(tweetWordList_cappos,tweetWordList):
    output=[]
    #identifies consecutive numbers in the sequence
    for k, g in groupby(enumerate(tweetWordList_cappos), lambda element: element[0]-element[1]):
        output.append(list(map(itemgetter(1), g)))
    count=0
    if output:        
        final_output=[output[0]]
        for first, second in (zip(output,output[1:])):
            if ((second[0]-first[-1])==2) & (tweetWordList[first[-1]+1] in prep_list):
                (final_output[-1]).extend([first[-1]+1]+second)
            else:
                final_output.append(second)
                #merge_positions.append(False)
    else:
        final_output=[]
    
    return final_output

In [63]:
#basically splitting the original NE_candidate text and building individual object from each text snippet
def build_custom_NE(phrase,prototype,feature_index,feature_value):
    #print("Enters")
    custom_NE= ne.NE_candidate(phrase)
    for i in range(6):
        custom_NE.set_feature(i,prototype.features[i])
    custom_NE.set_feature(feature_index,feature_value)
    if (feature_index== ne.is_csl) & (feature_value== True):
        custom_NE.set_feature(ne.start_of_sentence, False)
    return custom_NE

In [64]:
def abbrv_algo(ne_element):
    '''abbreviation algorithm 
    trailing apostrophe:
           |period:
           |     multiple letter-period sequence:
           |         all caps
           | non period:
           |     ?/! else drop apostrophe
    else:
        unchanged
    '''
    phrase= ne_element.phraseText
    #print(phrase)
    #since no further split occurs we can set remaining features now
    ne_element.set_feature(ne.capitalized, True)
    if ne_element.phraseText.isupper():
        ne_element.set_feature(ne.all_capitalized, True)
    else:
        ne_element.set_feature(ne.all_capitalized, False)
        
    abbreviation_flag=False
    p=re.compile(r'[^a-zA-Z\d\s]$')
    match_list = p.findall(phrase)
    if len(match_list)>0:
        if phrase.endswith('.'):
            p1= re.compile(r'([a-zA-Z][\.]\s*)')
            match_list = p1.findall(phrase)
            if len(match_list)>1:
                #print "Found abbreviation: "+phrase
                abbreviation_flag= True
            else:
                phrase= phrase[:-1]
        else:
            if not (phrase.endswith('?')|phrase.endswith('!')):
                phrase= phrase[:-1]
    else:
        p2=re.compile(r'[^a-zA-Z0-9_\s]')
        match_list = p.findall(phrase)
        if ((len(match_list)==0) & (phrase.isupper())):
            #print ("Found abbreviation: "+phrase)
            abbreviation_flag= True
        '''elif len(match_list)==1:
            print phrase'''
            
    #element= ne.NE_candidate(phrase.strip())
    ne_element.phraseText=phrase
    ne_element.reset_length()
    ne_element.set_feature(ne.abbreviation, abbreviation_flag)
    return ne_element
    

In [65]:
def punct_clause(NE_phrases):
    
    cap_phrases=NE_phrases.phraseText.strip()
    #print (cap_phrases)
    if (re.compile(r'[^a-zA-Z0-9_\s]')).findall(cap_phrases):
        #case of intermediate punctuations: handles abbreviations
        p1= re.compile(r'(?:[a-zA-Z0-9][^a-zA-Z0-9_\s]\s*)')
        match_lst = p1.findall(cap_phrases)
        

        index= (list( p1.finditer(cap_phrases) )[-1]).span()[1]
        p= re.compile(r'[^a-zA-Z\d\s]')
        match_list = p.findall(cap_phrases)

        p2=re.compile(r'[^a-zA-Z\d\s]$') #ends with punctuation

        if len(match_list)-len(match_lst)>0:
            if (p2.findall(cap_phrases)):
                #only strips trailing punctuations, not intermediate ones following letters
                cap_phrases = cap_phrases[0:index]+re.sub(p, '', cap_phrases[index:])
                NE_phrases.phraseText= cap_phrases
        
    
    #comma separated NEs
    #lst=filter(lambda(word): word!="", re.split('[,]', cap_phrases))
    start_of_sentence_fix=NE_phrases.features[ne.start_of_sentence]
    lst=list(filter(lambda word: word!="", re.split('[,]', cap_phrases)))
    if len(lst)>1:
        #print "True"
        final_lst= list(map(lambda phrase:build_custom_NE(phrase,NE_phrases,ne.is_csl,True), lst))
        final_lst[0].set_feature(ne.start_of_sentence, NE_phrases.features[ne.start_of_sentence])
    else:
        NE_phrases.set_feature(ne.is_csl,False)
        final_lst=[NE_phrases]
    
    #check abbreviation
    final_lst= list(map(lambda phrase: abbrv_algo(phrase), final_lst))

    
    #print(lst)
    return final_lst

In [66]:
def title_check(ne_phrase):
    title_flag=False
    words=ne_phrase.phraseText.split()
    for word in words:
        if word in cachedTitles:
            title_flag= True
            break
    ne_phrase.set_feature(ne.title,title_flag)
    return ne_phrase

In [67]:
'''def check(word):
    p=re.compile(r'([a-zA-Z])')
    match_lst = p.findall(word)
    if len(match_lst)==0:
            print word+ str(len(match_lst))
            return ""
    else:
        return word+" "'''

'def check(word):\n    p=re.compile(r\'([a-zA-Z])\')\n    match_lst = p.findall(word)\n    if len(match_lst)==0:\n            print word+ str(len(match_lst))\n            return ""\n    else:\n        return word+" "'

In [68]:
#%%timeit -o
def f(x,tweetWordList):

    #list1=map(lambda word: check(tweetWordList[word], word), x)
    list1=list(map(lambda word: tweetWordList[word]+" ", x[:-1]))
    phrase="".join(list1)+tweetWordList[x[-1]]
    '''if phrase=="":
        print "JUST_DIGIT_ERROR"
        phrase="JUST_DIGIT_ERROR"'''

    NE_phrase= ne.NE_candidate(phrase.strip())
    if 0 in x:
        NE_phrase.set_feature(ne.start_of_sentence,True)
    #else:
     #   NE_phrase.set_feature(ne.start_of_sentence,False)

    return NE_phrase

In [69]:
'''This is the main module. I am not explicitly writing it as a function as I am not sure what argument you are 
passing.However you can call this whole cell as a function and it will call the rest of the functions in my module
to extract candidates and features
'''

'''#reads input from the database file and converts to a dataframe. You can change this part accordingly and
#directly convert argument tuple to the dataframe'''

df = read_csv('/home/satadisha/Desktop/GitProjects/ELTweetTracker/Collection.csv', index_col='ID', header=0, encoding='utf-8')
#print df.columns.values.tolist()

#%%timeit -o
#module_capital_punct.main:
'''I am running this for 100 iterations for testing purposes. Of course you no longer need this for loop as you are
#running one tuple at a time'''

count=0
for index, row in df.iterrows():
    
    if count<250:
        #tweetText=unicode(row['Tweets']).encode('utf-8')
        tweetText=str(row['Tweets'])
        #tweetText=row['Tweets']
        print(tweetText)
        #capitalization module
        #if all words are capitalized:
        if tweetText.isupper():
            #print "All caps module"
            print ("All caps module")
        else:
            ne_List_final=[]
            #returns word list split at whitespaces; retains punctuation
            tweetSentenceList=list(filter (lambda sentence: len(sentence)>0, tweetText.split('\n')))
            for sentence in tweetSentenceList:
                tweetWordList= sentence.split()
                #print (tweetWordList)
                #returns position of words that are capitalized
                tweetWordList_cappos = list(map(lambda element : element[0], filter(lambda element : element[1][0].isupper(), enumerate(tweetWordList))))

                #tweetWordList_cappos = map(lambda (index, word) : index, filter(lambda (index, word) : word[0].isupper()|word[0].isdigit(), enumerate(tweetWordList)))
                #returns list with position of consecutively capitalized words
                output = consecutive_cap(tweetWordList_cappos,tweetWordList)
                #print (output)

                #consecutive capitalized phrases
                consecutive_cap_phrases_wStopwords=list(map(lambda x: f(x,tweetWordList), output))
                consecutive_cap_phrases=filter(lambda phrase: phrase.phraseText.lower() not in cachedStopWords ,consecutive_cap_phrases_wStopwords)
                #printList(flatten(consecutive_cap_phrases,[]))

                #implement the punctuation clause
                ne_List_pc=flatten(list(map(lambda NE_phrase: punct_clause(NE_phrase), consecutive_cap_phrases)),[])
                
                #implement title detection
                ne_List= list(map(lambda element: title_check(element), ne_List_pc))
                ne_List_final+=ne_List
                
            printList(ne_List_final)
            print ("")
        count+=1
    else:
        break



U.S.  Presidential Election series Trump, Clinton,Obama wins big, Rubio...?
[
U.S. Presidential Election 3 True True False False False False 
Trump 1 True None False False True False 
Clinton 1 True False False False True False 
Obama 1 True False False False True False 
Rubio 1 True None False False False False 
]

If Trump wins the presidential election, I'll make a logo for whoever RTs this tweet.
[
If Trump 2 True True False False False False 
RTs 1 True None False False False False 
]

Insanely Accurate' Election Model Has Been Wrong Only Once Since 1912. Guess Who It Says is The Winner?
[
Insanely Accurate' Election Model Has Been Wrong Only Once Since 10 True True False False False False 
Guess Who It Says 4 True None False False False False 
The Winner? 2 True None False False False False 
]

If you're eligible to vote in the US Presidential Election, make sure you hit up the polls TODAY 
[
US Presidential Election 3 True None False False False False 
TODAY 1 True None True Tru